# Tensor decomposition using Autograd with known features/factors

In a [previous post](), this post, I will compare an Autograd based implementation with a standard Tensorly based PARAFAC implementation.

### Imports

In [1]:
import autograd.numpy as np
import tensorly
from tensorly.decomposition import parafac
from autograd import multigrad

### Creating a tensor of size 2, 3, 4 from known factors

In [33]:
rank = 2
A_gt, B_gt, C_gt = np.random.rand(2, rank), np.random.rand(3, rank), np.random.rand(4, rank)

In [34]:
A_gt

array([[ 0.11827443,  0.63992102],
       [ 0.14335329,  0.94466892]])

In [35]:
B_gt

array([[ 0.52184832,  0.41466194],
       [ 0.26455561,  0.77423369],
       [ 0.45615033,  0.56843395]])

In [36]:
C_gt

array([[ 0.0187898 ,  0.6176355 ],
       [ 0.61209572,  0.616934  ],
       [ 0.94374808,  0.6818203 ],
       [ 0.3595079 ,  0.43703195]])

In [16]:
t = np.divide(np.arange(24).reshape(2, 3, 4).astype('float32'), 23)

In [17]:
t

array([[[ 0.        ,  0.04347826,  0.08695652,  0.13043478],
        [ 0.17391305,  0.2173913 ,  0.26086956,  0.30434781],
        [ 0.34782609,  0.39130434,  0.43478259,  0.47826087]],

       [[ 0.52173913,  0.56521738,  0.60869563,  0.65217394],
        [ 0.69565219,  0.73913044,  0.78260869,  0.82608694],
        [ 0.86956519,  0.9130435 ,  0.95652175,  1.        ]]], dtype=float32)

In [22]:
def khatri(A, B, C):
    return np.einsum('il,jl,kl->ijk',A,B,C)

In [40]:
tensor = khatri(A_gt, B_gt, C_gt)
tensor

array([[[ 0.16504986,  0.20148334,  0.23917099,  0.13815612],
        [ 0.30659446,  0.32481154,  0.36733682,  0.22777585],
        [ 0.22568039,  0.25743462,  0.29893014,  0.17836739]],

       [[ 0.24334473,  0.28745437,  0.33768199,  0.1980877 ],
        [ 0.45244781,  0.47443581,  0.53447118,  0.33327707],
        [ 0.33288775,  0.37130772,  0.42783745,  0.2581867 ]]])

In [47]:
B_known = B_gt[:,0]

### Using Tensorly to break it into rank 2 matrices

In [49]:
A, B, C = parafac(tensor, rank=2)

In [50]:
A

array([[-0.87198474,  0.02603646],
       [-1.27530509,  0.02846338]])

In [51]:
B

array([[-0.40776048, -1.06508155],
       [-0.71907855, -0.12387509],
       [-0.54353673, -0.77904427]])

In [52]:
C

array([[ 0.49362099,  0.41260138],
       [ 0.51472576, -0.65887442],
       [ 0.57862185, -1.21062624],
       [ 0.36192267, -0.33299139]])

In [53]:
khatri(A, B, C)

array([[[ 0.1640705 ,  0.20128758,  0.23930713,  0.13791972],
        [ 0.30818221,  0.32487124,  0.36671526,  0.22800874],
        [ 0.22558548,  0.25732154,  0.29879695,  0.17828958]],

       [[ 0.24418395,  0.2876415 ,  0.33759549,  0.19830158],
        [ 0.45121763,  0.47434957,  0.53489055,  0.3330733 ],
        [ 0.33301668,  0.37140515,  0.427931  ,  0.25825963]]])

### Cost function for GD based optimisation

In [54]:
def cost(A, B, C):
    pred = khatri(A, B, C)
    gt = t
    mask = ~np.isnan(t)
    error = (pred-gt)[mask].flatten()
    return np.sqrt((error**2).mean())

### Main autograd routine

In [55]:
def gd_decomposition(tensor, rank, m, n, o):
    mg = multigrad(cost, argnums=[0, 1, 2])
    lr = 0.01
    np.random.seed(0)
    m, n, o = t.shape
    a = np.random.randn(m, rank)
    b = np.random.randn(n, rank)
    c = np.random.randn(o, rank)

    for i in range(6000):
        del_a, del_b, del_c = mg(a, b, c)
        a-=lr*del_a
        b-=lr*del_b
        c-=lr*del_c
        if i%500==0:
            print(cost(a, b, c))
    return a, b, c

In [57]:
m, n, o = tensor.shape
a, b, c = gd_decomposition(tensor, 2, m, n, o)

1.59591131281
0.146098790525
0.0365466930582
0.0239552866637
0.0238907884086
0.0238298129994
0.0237694566118
0.0237098809544
0.0236512097445
0.0235935252784
0.02353687372
0.0234812709369


In [58]:
def gd_decomposition(tensor, rank, m, n, o, A_known=None, B_known=None, C_known=None):
    def set_known(A, W):
        mask = ~np.isnan(W)
        A[:,:mask.shape[1]][mask] = W[mask]
        return A
    mg = multigrad(cost, argnums=[0, 1, 2])
    lr = 0.01
    np.random.seed(0)
    m, n, o = t.shape
    a = np.random.randn(m, rank)
    b = np.random.randn(n, rank)
    c = np.random.randn(o, rank)
    
    
    for i in range(6000):
        del_a, del_b, del_c = mg(a, b, c)
        a-=lr*del_a
        b-=lr*del_b
        c-=lr*del_c
        if A_known is not None:
            a = set_known(a, A_known)
        if B_known is not None:
            b = set_known(b, B_known)
        if C_known is not None:
            c = set_known(c, C_known)

        if i%500==0:
            print(cost(a, b, c))
    return a, b, c

In [66]:
gd_decomposition(tensor, 2, m, n, o, A_known=A_gt[:,:1], B_known=B_gt[:,:1], C_known=C_gt[:,:1])

IndexError: too many indices for array

### Decomposing with missing entries

In [15]:
t[0, 0, 0] = np.NAN
t[1, 1, 2] = np.NAN
t

array([[[ nan,   1.,   2.,   3.],
        [  4.,   5.,   6.,   7.],
        [  8.,   9.,  10.,  11.]],

       [[ 12.,  13.,  14.,  15.],
        [ 16.,  17.,  nan,  19.],
        [ 20.,  21.,  22.,  23.]]], dtype=float32)

In [16]:
rank = 2
lr = 0.01

np.random.seed(1)
m, n, o = t.shape
a = np.random.randn(m, rank)
b = np.random.randn(n, rank)
c = np.random.randn(o, rank)

for i in range(6000):
    del_a, del_b, del_c = mg(a, b, c)
    a-=lr*del_a
    b-=lr*del_b
    c-=lr*del_c
    if i%500==0:
        print(cost(a, b, c))

14.3657653138
11.2554542723
0.391440064205
0.320089741632
0.279150824088
0.254263517167
0.235274595444
0.217495424995
0.197602923558
0.218594628293
0.223672460718
0.225830360065


In [17]:
np.round(khatri(a, b, c))

array([[[ -0.,   1.,   2.,   3.],
        [  4.,   5.,   6.,   7.],
        [  8.,   9.,  10.,  11.]],

       [[ 12.,  13.,  14.,  15.],
        [ 16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.]]])

### What if we used a different seed?

In [18]:

np.random.seed(0)
m, n, o = t.shape
a = np.random.randn(m, rank)
b = np.random.randn(n, rank)
c = np.random.randn(o, rank)

for i in range(6000):
    del_a, del_b, del_c = mg(a, b, c)
    a-=lr*del_a
    b-=lr*del_b
    c-=lr*del_c
    if i%500==0:
        print(cost(a, b, c))

13.1375238648
1.42377956103
1.26196020429
0.47681451677
0.476038335842
0.475054930056
0.47360718886
0.471433455007
0.468181835873
0.463411604108
0.456562523642
0.446768595735


In [19]:
np.round(khatri(a, b, c))

array([[[  2.,   1.,   2.,   3.],
        [  5.,   5.,   6.,   6.],
        [  9.,   9.,  10.,  10.]],

       [[ 12.,  13.,  14.,  15.],
        [ 16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.]]])

Okay. Not so good result this time!